In [17]:
import numpy as np
import pandas as pd
import yfinance as yf


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [18]:
df = pd.read_csv('asset_returns.csv', names=[i for i in range(83)])
daily_returns = df.fillna(0)
daily_returns.head()

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,-0.005763,-0.026772,-0.087660,0.026042,-0.003650,-0.074595,-0.057377,-0.084338,-0.084821,0.081127,...,-0.049128,-0.094203,0.000000,-0.083417,0.109715,-0.025634,0.015899,-0.014392,0.042297,-0.010116
1,0.058059,0.113269,0.175373,0.096116,0.076657,0.120327,0.010870,0.102639,0.154634,0.130871,...,-0.029639,0.160000,-0.083081,-0.021158,-0.019817,0.030551,-0.021737,0.003431,0.004292,-0.065693
2,-0.037039,0.035610,-0.026984,-0.077318,-0.144086,-0.012513,-0.053763,-0.031031,-0.013519,-0.117281,...,-0.014870,-0.006897,-0.178512,0.004320,-0.009998,-0.049396,0.000881,-0.068494,-0.036105,0.003123
3,0.055546,-0.002807,0.093801,0.044626,0.089131,0.062302,0.011364,0.067736,0.018844,0.013110,...,0.052459,-0.055556,-0.051674,0.058668,-0.050943,0.096973,0.014222,0.087317,0.034290,0.004675
4,0.037787,0.035890,0.021626,0.163150,0.064524,0.053678,0.024719,0.035753,-0.121480,0.033673,...,0.046823,0.077206,0.036358,0.023387,-0.172145,0.128671,0.077061,0.014368,0.068786,0.062015


In [19]:
def optimal_weights(rets, target_return):
    mean_return = rets.mean().to_numpy().reshape(-1, 1)
    daily_cov = rets.cov().to_numpy()
    e = np.ones(len(mean_return)).reshape(-1, 1)
    result = []
    for r_p in target_return:
        
        top_row = np.hstack((daily_cov, -mean_return, -e))
        middle_row = np.hstack((-mean_return.T, np.zeros((1, 1)), np.zeros((1, 1))))
        bottom_row = np.hstack((-e.T, np.zeros((1, 1)), np.zeros((1, 1))))

        A = np.vstack((top_row, middle_row, bottom_row))
        b = np.vstack((np.zeros((len(mean_return), 1)), -r_p, -1))

        # Solve the linear system
        try:
            x = np.linalg.solve(A, b)
            result.append(x.flatten())
        except np.linalg.LinAlgError as err:
            print(f"Cannot solve linear system for target return {r_p}: {err}")
            continue

    results_df = pd.DataFrame(result, columns=[f"Asset {i+1}" for i in range(len(mean_return))] + ["Lambda", "Mu"])
    results_df.set_index(target_return, inplace=True)
    return results_df

### Actual average returns, backtesting:

$ \overline{r}^\top w $ - actual average returns

$ w^\top \Sigma w^\top $ - portfolio covariance 

where $w$ denote the optimal portfolio, $\overline{r}$ denote the average return over
the out-of-sample period, and $\Sigma$ denote the out of sample covariance matrix.


In [20]:
def backtesting(optimal_weights, OOS_rets, target_return):
    mean_return_OOS = OOS_rets.mean().to_numpy().reshape(-1, 1)
    daily_cov_OOS = OOS_rets.cov().to_numpy()
    arr = optimal_weights.to_numpy()
    res = []

    for index, row in enumerate(arr):
        weights = row[:-2]
        targ_ret = target_return[index]
        act_ave_return = (mean_return_OOS.T @ weights).item()
        pf_cov = weights.T @ daily_cov_OOS @ weights
        res.append([targ_ret, act_ave_return, pf_cov])
    
    return np.array(res)

# Main

In [21]:
target_return = np.linspace(0, 0.1, 21)

results_dict = {}
N = len(range(0, daily_returns.shape[0]-100, 12))
for i in range(0, daily_returns.shape[0]-100, 12):
    index = int(i/12)
    start, mid, end = i, i+100, i +112
    daily_returns_IS = daily_returns.iloc[start:mid,:]
    daily_returns_OOS = daily_returns.iloc[mid:end]
    df_weights = optimal_weights(daily_returns_IS, target_return)
    df_act_returns = backtesting(df_weights, daily_returns_OOS, target_return)
    results_dict[index] = pd.DataFrame(df_act_returns, columns=['target_return', 'Actual Average Return', 'Portfolio Covariance']).set_index('target_return')

In [22]:
import time

def plot(tret):
    d = []
    e = []
    for key, item in results_dict.items():
        d.append(item['Actual Average Return'].loc[tret])
        e.append(item['Portfolio Covariance'].loc[tret])

    fig = px.line(d)
    fig.add_hline(y=tret, line_dash="dash", line_color="red")
    fig.show()

for t in target_return.tolist():
    plot(t)
